<a href="https://colab.research.google.com/github/Navya123801/Churn-Modelling/blob/main/AI_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers sentence-transformers


In [5]:
from sentence_transformers import SentenceTransformer, util

faq = {
    "latest smartphone specs": "Our latest smartphone features a 6.5-inch OLED display, 128GB storage, and a 48MP camera.",
    "order tracking": "You can track your order using the tracking link sent to your email after purchase.",
    "return policy": "Returns accepted within 30 days with the original receipt and packaging.",
    "payment methods": "We accept credit cards, debit cards, PayPal, and Apple Pay.",
    "warranty": "All products come with a 1-year limited warranty covering manufacturing defects."
}

embedder = SentenceTransformer('all-MiniLM-L6-v2')

faq_questions = list(faq.keys())
faq_embeddings = embedder.encode(faq_questions, convert_to_tensor=True)

def find_best_faq_answer(user_query):
    query_embedding = embedder.encode(user_query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, faq_embeddings, top_k=1)
    best_match_idx = hits[0][0]['corpus_id']
    score = hits[0][0]['score']
    if score > 0.5:
        return faq[faq_questions[best_match_idx]]
    return None


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

# Store chat history IDs for multi-turn conversation
chat_history_ids = None


In [9]:
print("Chatbot is ready! Type 'exit' or 'quit' to stop.")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break

    # Check FAQ first
    faq_answer = find_best_faq_answer(user_input)
    if faq_answer:
        print("Bot (FAQ):", faq_answer)
        continue

    # If no FAQ answer, use the LLM conversational model
    # Encode user input + chat history
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        bot_input_ids = new_input_ids

    # Generate response
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    bot_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    print("Bot:", bot_response)


Chatbot is ready! Type 'exit' or 'quit' to stop.
You: "Tell me about the warranty and how to track orders.
Bot (FAQ): All products come with a 1-year limited warranty covering manufacturing defects.
You: exit
Bot: Goodbye!
